In [1]:
import os

def convertToTxt(path):
    for i in os.listdir(path):
        os.rename(dir + "/" + i, dir + "/" + i + ".txt")
        


In [2]:
#get all files in directory
def filesInDirectory(path):
    files = []
    for i in os.listdir(path):
        files.append(path  + "/" + i)
    return files


In [3]:
#Now using pandas we will read each data trail and save it to an array of datasets that contain the eeg singal for each channel
import pandas as pd
import numpy as np


In [4]:

def collectTrialData(TrialNumber):
    with open(SubjectTrialDataset[TrialNumber]) as f:
        lines = f.readlines()

    del lines[0:4]
    count = 0
    valsPerCol = []
    channels = [[0]* 256 for _ in range(64)]

    i = 0
    for line in lines:
        if(line[0] == "#"):   
            line = line.replace("#", "")
            line = line.replace("\n", "")
            line = line.replace(",", "")
            line = line.lstrip()
            channels[i][0] = line
            i += 1  
        #print(count)
    #print(channels)
    currChan = -1
    for line in lines:
        count += 1
        count = count % 257
        if(count == 1):
            #print(valsPerCol)
            currChan += 1
            z = 0
        #Data from cols
        
        if(count != 1):
            z += 1
            work = line.split(" ")
            #print(z-1)
            b = 0
            for w in work:
                b += 1
                b = b % 4
                if(b == 0):
                
                    channels[currChan][z-1] = w

    #Channel[channelNumber][Sample Number] = eegSignal
    #print(channels[0][0])
    return channels


In [5]:
#This method is to store the data from subject into a dataset we can view
path = "Alcoholics/co2a0000364"
#This contains an array of each trial for a subject
SubjectTrialDataset = filesInDirectory(path)

#Data set is stored in a matrix where first value is column and second value is sample number
# FP1 FP2 ...
# -1  -3.2
# ..  .. 

print(SubjectTrialDataset[1])
#This method generates all trial data for set person
listOfAllTrials = []
for i in range(30):
    listOfAllTrials.append(collectTrialData(i))
val = listOfAllTrials[1]
print(val[0][0])




FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Alcoholics/co2a0000364'